In [1]:
import awswrangler as wr
import pandas as pd
import evaluate
import numpy as np

from transformers import (
    AutoTokenizer,
    DataCollatorWithPadding,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
)

from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict

In [2]:
pd.options.mode.copy_on_write = True

In [3]:
df = wr.s3.read_parquet(path="s3://amazon-reviews-eafit/sample/")

In [4]:
label2id = {"negative": 0, "neutral": 1, "positive": 2}
id2label = {0: "negative", 1: "neutral", 2: "positive"}
df["labels"] = df["sentiment"].map(label2id)

In [5]:
df_sentiment = df[["review_body", "sentiment", "labels"]]

In [6]:
df_temp, df_test = train_test_split(df_sentiment, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_temp, test_size=0.5, random_state=42)

In [7]:
dataset_test = Dataset.from_pandas(df_test)
dataset_val = Dataset.from_pandas(df_val)
dataset_train = Dataset.from_pandas(df_train)

dataset_dict = DatasetDict(
    {"train": dataset_train, "validation": dataset_val, "test": dataset_test}
)

In [8]:
model = "distilbert/distilbert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model)

/home/camilo/proyecto-integrador-semestre-2/.venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [9]:
def preprocess_function(examples):
    return tokenizer(examples["review_body"], truncation=True)

In [10]:
tokenized_data_train = dataset_dict["train"].map(preprocess_function, batched=True)
tokenized_data_validation = dataset_dict["validation"].map(
    preprocess_function, batched=True
)
tokenized_data_test = dataset_dict["test"].map(preprocess_function, batched=True)

Map:   0%|          | 0/399693 [00:00<?, ? examples/s]

Map:   0%|          | 0/399694 [00:00<?, ? examples/s]

Map:   0%|          | 0/199847 [00:00<?, ? examples/s]

In [11]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [12]:
accuracy = evaluate.load("accuracy")

In [13]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [14]:
model = AutoModelForSequenceClassification.from_pretrained(
    model, num_labels=3, id2label=id2label, label2id=label2id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
small_train_dataset = tokenized_data_train.select(range(4000))
small_eval_dataset = tokenized_data_validation.select(range(4000))

In [16]:
training_args = TrainingArguments(
    output_dir="../data/prediction_sentiment",
    learning_rate=2e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.003406,0.900000
2,No log,0.968266,0.900000
3,No log,0.938913,0.900000
4,No log,0.913134,0.900000
5,No log,0.889862,0.900000
6,No log,0.869634,0.900000
7,No log,0.852722,0.900000
8,No log,0.839562,0.900000
9,No log,0.830647,0.900000
10,No log,0.826095,0.900000


[[-0.07034697 -0.06512833  0.09396879]
 [-0.08964043 -0.10036284  0.14079048]
 [-0.08319017 -0.0696836   0.09275185]
 [-0.06414641 -0.10149805  0.12201326]
 [-0.09208728 -0.11780275  0.03106743]
 [-0.04814252 -0.12362385  0.0518225 ]
 [-0.09157604 -0.10261454  0.10668561]
 [-0.04655125 -0.10129008  0.11398774]
 [-0.06767966 -0.10876579  0.09050222]
 [-0.107526   -0.08147936  0.1240765 ]]
[2 0 2 2 2 2 2 2 2 2]
[[-0.10268004 -0.09105799  0.14089754]
 [-0.12362011 -0.12548278  0.18410753]
 [-0.10835928 -0.09061238  0.12833878]
 [-0.08618977 -0.11635546  0.15498936]
 [-0.12620082 -0.14373538  0.07994751]
 [-0.07624983 -0.14330585  0.09357709]
 [-0.12474862 -0.12781502  0.14986454]
 [-0.0656674  -0.11412975  0.14597331]
 [-0.11157286 -0.14428517  0.14732778]
 [-0.13800226 -0.11187408  0.16831607]]
[2 0 2 2 2 2 2 2 2 2]
[[-0.12901244 -0.11269853  0.18216753]
 [-0.15290445 -0.148538    0.22164413]
 [-0.13012674 -0.11054542  0.15793431]
 [-0.10418148 -0.13131537  0.18383773]
 [-0.15701336 -0.1

In [20]:
df_results = pd.DataFrame(trainer.state.log_history)

In [23]:
df_results.to_parquet("../data/prediction_sentiment_results/metrics.parquet")

# Test

In [31]:
small_test_dataset = tokenized_data_test.select(range(50))

In [32]:
predictions_test = trainer.predict(small_test_dataset)

[[-0.25960073 -0.20126009  0.35329914]
 [-0.20641065 -0.27886903  0.2990285 ]
 [-0.22442305 -0.23175992  0.41484722]
 [-0.2592337  -0.28604758  0.37312222]
 [-0.14394376 -0.16613027  0.20483293]
 [-0.28855333 -0.26329023  0.3183465 ]
 [-0.24456218 -0.21102923  0.33713728]
 [-0.31210604 -0.23386143  0.3391418 ]
 [-0.21083833 -0.2646261   0.34310266]
 [-0.2828176  -0.22596319  0.2903264 ]
 [-0.23851496 -0.2348367   0.33882952]
 [-0.18507767 -0.2209858   0.3215231 ]
 [-0.26124606 -0.34718725  0.3388014 ]
 [-0.15659906 -0.14399435  0.24770403]
 [-0.26132092 -0.29749846  0.31240577]
 [-0.23109068 -0.22583587  0.29862145]
 [-0.23882306 -0.21832752  0.2835646 ]
 [-0.2242401  -0.19587764  0.38308346]
 [-0.28589264 -0.26042828  0.38531306]
 [-0.21789905 -0.2553931   0.348731  ]
 [-0.2943241  -0.25461313  0.35278887]
 [-0.19543068 -0.23868348  0.26050434]
 [-0.2233388  -0.32181022  0.34461638]
 [-0.23570442 -0.23283076  0.37417972]
 [-0.33094805 -0.2869413   0.35011443]
 [-0.20142892 -0.15367495

In [33]:
df_result_test = pd.DataFrame(predictions_test.metrics)

df_result_test.to_parquet("../data/prediction_sentiment_results/metrics_test.parquet")

{'test_loss': 0.8799773454666138,
 'test_accuracy': 0.78,
 'test_runtime': 0.8222,
 'test_samples_per_second': 60.814,
 'test_steps_per_second': 1.216}